<a href="https://colab.research.google.com/github/yujungha/Data_Analysis/blob/main/16_2%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
def dateformat(y=0,m=0,d=0,h=0,mm=0,s=0):
    return "%d-%02d-%02d %02d:%02d:%02d"%(y,m,d,h,mm,s)

In [ ]:
# txt인 경우 (iphone)
f = open('Talk_2021.4.29 21_54-1.txt', mode='r', encoding='utf-8')

file_name = f.readline()
save_date_re = re.compile('저장한 날짜 : ([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2})')
m = save_date_re.match(f.readline())
save_date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))

f.readline() # 공백 제거
f.readline() # 공백 제거
f.readline() # 공백 제거

history = []
histline = ''
date = None

date_re = re.compile('([0-9]{4})년 ([0-9]{1,2})월 ([0-9]{1,2})일 ((월|화|수|목|금|토|일)요일)')
invite_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}): ((.+)님이 .+ 초대했습니다.)')
exit_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}): ((.+)님이 나갔습니다.)')
chat_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}), (.+) : (.+)')

chat_temp = ''

i=0
while True:
    line = f.readline()
    if not line: 
        break
    # print(line, end='')

    # chat
    m = chat_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
        user = m[6]
        content = m[7]
        history.append([date, user, content])
        continue
    # date
    m = date_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]))
        # print('aaaaaaaaaaaa' ,history[-1][2])
        continue
    # invite
    m = invite_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
        user = m[7]
        content = m[6]
        history.append([date, user, content])
        continue
    # exit
    m = exit_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
        user = m[7]
        content = m[6]
        history.append([date, user, content])
        continue
    # \n chat
    history[-1][2] += line
df = pd.DataFrame(history, columns=['Date', 'User', 'Message'])

In [ ]:
df

,Date,User,Message
0,2019-05-19 18:46:00,19 김은하,19 김은하님이 누리고라스님과 유정하님을 초대했습니다.
1,2019-05-19 18:46:00,19 김은하,애들아
2,2019-05-19 18:47:00,19 김은하,조이름생각해바
3,2019-05-19 18:47:00,유정하,티미룸 괜찮은데..
4,2019-05-19 18:47:00,유정하,조이룸..
...,...,...,...
4712,2021-04-29 18:29:00,유정하,센스 쩌렀찌
4713,2021-04-29 18:36:00,누리고라스,웅웅
4714,2021-04-29 18:36:00,누리고라스,개쩔어
4715,2021-04-29 21:54:00,유정하,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ


In [ ]:
a = df['Message'].values
b = []
for i in range(len(a) - 1):
  b.append([a[i], a[i + 1]])
b

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-4ylthykr
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-4ylthykr
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=5ec9e4ebae448bbe3d5f3a373cfac90de6de964fd2bb41b6c8186c37ec4bf57f
  Stored in directory: /tmp/pip-ephem-wheel-cache-tkuet8ag/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker
a = np.array(df[['User', 'Message']].values)
# a = [['19 김은하', '19 김은하님이 누리고라스님과 유정하님을 초대했습니다.'],
#     ['19 김은하', '애들아'],
#     ['19 김은하', '조이름생각해바']]
i = 0
while i != len(a)-1:
  print('%d / %d'%(i, len(a)))
  if a[i][0] == a[i+1][0]:
    a[i][1] += ' '+a[i+1][1]
    a = np.delete(a, i+1, 0)
  else:
    a[i][1] = spell_checker.check(a[i][1]).checked
    i += 1

a = a.T[1]

b = []
for i in range(len(a) - 1):
  b.append([a[i], a[i + 1]])
b

0 / 4717
0 / 4716
0 / 4715
1 / 4715
1 / 4714
2 / 4714
2 / 4713
2 / 4712
2 / 4711
3 / 4711
4 / 4711
5 / 4711
5 / 4710
6 / 4710
7 / 4710
8 / 4710
9 / 4710
9 / 4709
10 / 4709
11 / 4709
11 / 4708
12 / 4708
13 / 4708
13 / 4707
14 / 4707
15 / 4707
15 / 4706
15 / 4705
16 / 4705
16 / 4704
17 / 4704
18 / 4704
18 / 4703
18 / 4702
18 / 4701
18 / 4700
18 / 4699
19 / 4699
20 / 4699
21 / 4699
21 / 4698
22 / 4698
22 / 4697
23 / 4697
24 / 4697
25 / 4697
26 / 4697
27 / 4697
28 / 4697
29 / 4697
30 / 4697
31 / 4697
32 / 4697
32 / 4696
32 / 4695
32 / 4694
33 / 4694
34 / 4694
34 / 4693
34 / 4692
35 / 4692
36 / 4692
37 / 4692
37 / 4691
38 / 4691
38 / 4690
39 / 4690
39 / 4689
39 / 4688
40 / 4688
40 / 4687
41 / 4687
42 / 4687
42 / 4686
43 / 4686
43 / 4685
44 / 4685
44 / 4684
44 / 4683
44 / 4682
45 / 4682
46 / 4682
47 / 4682
47 / 4681
48 / 4681
49 / 4681
50 / 4681
51 / 4681
52 / 4681
52 / 4680
53 / 4680
53 / 4679
54 / 4679
54 / 4678
55 / 4678
56 / 4678
57 / 4678
58 / 4678
58 / 4677
59 / 4677
60 / 4677
60 / 467

In [ ]:
import random

while True:
  Q = input("질문 : ")
  l1 = []
  l2 = []
  l3 = []

  for i in range(len(b)):
    if Q == b[i][0]:
      # print('[완벽일치]', b[i][0])
      l1.append(b[i][1])
    elif b[i][0].find(Q) != -1:
      # print('[포함]',b[i][0])
      l2.append(b[i][1])
    # [포함-2] 채팅
    else:
      n = 0
      for q in Q.split(' '):
        if b[i][0].find(q) != -1:
          n += 1
      if n > 0:
        # print('[포함-%d]'%n,b[i][0])
        l3.append(b[i][1])

  if len(l1) != 0:
    print('답변 :',random.choice(l1))
  elif len(l2) != 0:
    print('답변 :',random.choice(l2))
  elif len(l3) != 0:
    print('답변 :',random.choice(l3))
  else:
    print('답변 : 맞아맞아')
  print('\n=================================================\n')